# IPL Powerplay Score Prediction
![IPL banner](https://cricketaddictor.gumlet.io/wp-content/uploads/2021/02/153757448_173638941016448_6980867142752435675_n.jpg?compress=true&quality=80&w=1920&dpr=2.6)

It's COVID lockdown time. Everyone is worried about the increasing COVID cases, Work From Home (WFH) environment. No way to go out, no parties, no outing, no vacation plans, no gatherings ...... 

Government announces Lockdown : 1.0, 2.0, 3.0, 4.0 .....

At that time every cricket fan had a question. 
> **"Do new Govt. rules in lockdown 4.0 pave the way for IPL 2020 ????"**



Finally IPL 2020 happend. 
* **Even pandamic situation can't derail the IPL juggernaut.**
* IPL 20202 earned revenue of : 4000 crores with 
  *  35% reduced cost and 
  *  25% increase in viewership. 

As a cricket fan I watch all the matches, during that time I observed that **"Powerplay plays a mojor role"** which is very Important in Team score.

## What is powerplay in IPL & it's Importance???
* In summary "powerplay" means fielding restrictions in 1<sup>st</sup> 6-overs.
  * means, only 2-fielders can stay outside the Inner-circle
  * After powerplay, upto 5-fielders can stay outside inner circle & 4-fielders must remain inside the inner circle.
* Which makes the **batting compatively easy.**
* Also **it's a trap for the batsmen**, as this will get them to take into a risk and loose their wickets in 1<sup>st</sup> 6-overs

## Effect of "Powerplay" in IPL matches??
* Powerplay overs are considered as pillars of any teams  victory.
* 75% - of winning chance of the team depends on the Powerplay score.
* So, every team's expectation from the top 3-batsmen is **"START THE INNINGS BIG"**

In this blog post I am going to predict the score of an IPL match at the end of powerplay.

In [ ]:
import requests,zipfile,io,json
from pathlib import Path

#from sklearn.preprocessing import LabelEncoder
from collections import OrderedDict

import pandas as pd
import seaborn as sns
import numpy as np

import torch 

from matplotlib import pyplot as plt
%matplotlib inline

# **Variable Declaration:**

In [ ]:
epochs      = 100
lr          = 1e-1
split_ratio = (70.0,30.0)
train_bs    = 100   # Then test_bs = 2*train_bs
pltName     = 'test6.png'
changeDescription   = 'With L1 Loss'

seed = 143
torch.manual_seed(seed)

In [ ]:
dataset_path = Path(Path.cwd(),'dataset')
snippets_path = Path(Path.cwd(),'snippets')
filename = 'all_matches.csv'
recordedMetrics = 'Recorded_Metrics.csv'

json_path = Path.joinpath(dataset_path, 'label_encode.json')

save_files = False
record = True

# **1.Loading the Data Set:** 

Dataset will be available at : https://cricsheet.org/downloads/ipl_male_csv.zip

In [ ]:
def getCsvFile(url="https://cricsheet.org/downloads/ipl_male_csv2.zip"):
    res = requests.get(url, stream=True)
    if res.status_code == 200:
        print('### Downloading the CSV file')
        z = zipfile.ZipFile(io.BytesIO(res.content))
        if filename in z.namelist():
            z.extract(filename, dataset_path)
            print('### Extracted %s file' % filename)
        else:
            print('### %s : File not found in ZIP Artifact' % filename)

def downloadDataset():
    if not dataset_path.exists():
        Path.mkdir(dataset_path)
        print('### Created Dataset folder')
        getCsvFile()
    elif dataset_path.exists():
        files = [file for file in dataset_path.iterdir() if file.name ==
                 'all_matches.csv']
        if len(files) == 0:
            getCsvFile()
        else:
            print('### File already extracted in given path')
downloadDataset()

# **2. Data Frames:** 

In [ ]:
csv_file=Path.joinpath(dataset_path, filename)

df = pd.read_csv(csv_file,parse_dates=['start_date'],low_memory=False)
df_parsed = df.copy(deep=True)
print(df.shape)
df.head()


# **3. Exploratory Data Analysis (EDA) / Data Processing:**
## 3.1: Identify "null" values in Dataset:

In [ ]:
df.isnull().sum()

## 3.2: Cleaning Steps:
1. Here columns 'season' and 'start_date'  are not needed for our prediction. ```So we can drop the columns 'season' and 'start_date' from the dataset.```
2. Delete Non-existing teams : 
```
       'Kochi Tuskers Kerala' 'Pune Warriors','Rising Pune Supergiants', 'Rising Pune Supergiant','Gujarat Lions'
```
3. Replace the old team names with new team name:
```
      'Delhi Daredevils'  --> 'Delhi Capitals'
      'Deccan Chargers'   --> 'Sunrisers Hyderabad'
      'Punjab Kings'      --> 'Kings XI Punjab'
```

4. Correct the venue column with unique names. In this dataset same stadium is being represented as in multiple ways. So identify those and rename.
```
      ['M Chinnaswamy Stadium', 'M.Chinnaswamy Stadium']
      ['Brabourne Stadium', 'Brabourne Stadium, Mumbai']
      ['Punjab Cricket Association Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium']
      ['Wankhede Stadium', 'Wankhede Stadium, Mumbai']
      ['Rajiv Gandhi International Stadium, Uppal', 'Rajiv Gandhi International Stadium']
      ['MA Chidambaram Stadium, Chepauk','MA Chidambaram Stadium',      'MA Chidambaram Stadium, Chepauk, Chennai']
```

5. Rename the column names (for easy coding)
```
      'striker'     --> 'batsmen'
      'non-striker' --> 'batsmen_nonstriker'  (This column is not required)
      'bowler'      --> 'bowlers'
```
      
6. Create a columns "Total_score" : which reflects the runs through bat and extra runs through wides,byes,noballs,legbyes...etc.  
Hence we can drop columsn ```['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed']```

In [ ]:
df_parsed.drop(columns=['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed','season','start_date'],axis=1,inplace=True)
df_parsed.info()

In [ ]:
non_exist_teams = ['Kochi Tuskers Kerala',
                    'Pune Warriors',
                    'Rising Pune Supergiants',
                    'Rising Pune Supergiant',
                    'Gujarat Lions']
mask_bat_team = df_parsed['batting_team'].isin(non_exist_teams)
mask_bow_team = df_parsed['bowling_team'].isin(non_exist_teams)
df_parsed = df_parsed[~mask_bat_team]
df_parsed = df_parsed[~mask_bow_team]

In [ ]:
df_parsed.loc[df_parsed.batting_team ==
                'Delhi Daredevils', 'batting_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.batting_team == 'Deccan Chargers',
                'batting_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.batting_team ==
                'Punjab Kings', 'batting_team'] = 'Kings XI Punjab'

df_parsed.loc[df_parsed.bowling_team ==
                'Delhi Daredevils', 'bowling_team'] = 'Delhi Capitals'
df_parsed.loc[df_parsed.bowling_team == 'Deccan Chargers',
                'bowling_team'] = 'Sunrisers Hyderabad'
df_parsed.loc[df_parsed.bowling_team ==
                'Punjab Kings', 'bowling_team'] = 'Kings XI Punjab'



In [ ]:
df_parsed.venue.unique()

In [ ]:
list(filter(lambda x : 'chidam' in x.lower(),list(df_parsed.venue.unique())))
    

In [ ]:
df_parsed.loc[df_parsed.venue == 'M.Chinnaswamy Stadium',
                'venue'] = 'M Chinnaswamy Stadium'
df_parsed.loc[df_parsed.venue == 'Brabourne Stadium, Mumbai',
                'venue'] = 'Brabourne Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium, Mohali',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association IS Bindra Stadium',
                'venue'] = 'Punjab Cricket Association Stadium'
df_parsed.loc[df_parsed.venue == 'Wankhede Stadium, Mumbai',
                'venue'] = 'Wankhede Stadium'
df_parsed.loc[df_parsed.venue == 'Rajiv Gandhi International Stadium, Uppal',
                'venue'] = 'Rajiv Gandhi International Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk',
                'venue'] = 'MA Chidambaram Stadium'
df_parsed.loc[df_parsed.venue == 'MA Chidambaram Stadium, Chepauk, Chennai',
                'venue'] = 'MA Chidambaram Stadium'
df_parsed.loc[df_parsed.venue == 'Arun Jaitley Stadium','venue'] = 'Arun Jaitley Stadium, Delhi'
df_parsed.loc[df_parsed.venue == 'Punjab Cricket Association Stadium, Mohali','venue'] = 'Punjab Cricket Association Stadium'


In [ ]:
df_parsed = df_parsed.rename(columns={
                                 'striker': 'batsmen', 'non_striker': 'batsmen_non_striker', 'bowler': 'bowlers'})

In [ ]:
df_parsed.info()

In [ ]:
df_parsed['Total_score'] = df_parsed.runs_off_bat + df_parsed.extras

#df_parsed.drop(columns=['wides', 'noballs', 'byes', 'legbyes', 'penalty', 'wicket_type','other_wicket_type', 'other_player_dismissed'], axis=1, inplace=True)

In [ ]:
df_parsed.head()

## 3.3: Select required Data:
   Here as we are predicting the IPL match **"Powerplay"** score, we can drop the rest of the balls and run detail through, 
   * by selecting the 1<sup>st</sup> 6-Overs details and  
   * with Innings (1 and 2) : as there is no 3<sup>rd</sup> or 4<sup>th</sup> innings happen in powerplay


In [ ]:
df_parsed = df_parsed[(df_parsed.ball < 6.0) & (df_parsed.innings < 3)]

In [ ]:
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path, '1_parseNcleaned_Dataset.csv'), index=False)

print('### Total {} : venue details present '.format(
    len(df_parsed.venue.unique())))
print('### Total {}  : Batting teams are there'.format(
    len(df_parsed.batting_team.unique())))
print('### Total {}  : Bowlling teams are there'.format(
    len(df_parsed.bowling_team.unique())))
print('### Batting teams are : {}'.format(df_parsed.batting_team.unique()))
print('### Bowling teams are : {}'.format(df_parsed.bowling_team.unique()))
print('### Shape of data frame after initial cleanup :{}'.format(df_parsed.shape))


In [ ]:
#df_parsed = pd.read_csv(Path.joinpath(dataset_path, '1_parseDataset.csv'))
#df_parsed.shape

# **4. Encoding:** 

Dataframe column values of Dtype Object:
Till now as a initial step in step-2,
* We cleaned our dataset with all the "null" values and 
* filtered the columns/rows data (which are not used for prediction) and
* Added the required column with values (like total_score) to make the dataset clean.

As you can see above, our cleaned dataset is having 13 columns of multiple Dtype like int64 and float64 and object.

So next what I am going to do is trying to convert all these multiple Dtypes into a single Dtype, to train my model.

## 4.1:  Encode "batsmen" and "bowlers" column values:
   * Here we can see, few players who can bat as well as bowl.
    Means same player will be listed as a batsmen and as well as bowler.
    * So to make the prediction properly, I am creating one Dataframe with all the players name it as "players_df", which I use for encoding the players with some value to identify.
* For inference going further, I create a dictionary with all these encoded values 

Use "sklearn label encoder" module to encode the dataframe

In [ ]:
players_df = pd.DataFrame(np.append(
                            df_parsed.batsmen.unique(), df_parsed.bowlers.unique()),
                            columns=['Players']
                        )
players_df.head()

In [ ]:
label_encode_dict = {}

#le = LabelEncoder()
#le.fit(players_df.Players)
#Players_e = le.transform(players_df.Players)
#Players_e_inv = le.inverse_transform(Players_e)

dct = dict(enumerate(players_df.Players.astype('category').cat.categories))
label_encode_dict['Players'] = dict(zip(dct.values(),dct.keys()))

#label_encode_dict

## 4.2: Encode "vennue" and "batting_team" and "bowling_team" column values:

In [ ]:
#df_parsed.select_dtypes('object').columns
for col in ['venue', 'batting_team', 'bowling_team']:
    dct = dict(enumerate(df_parsed[col].astype('category').cat.categories))
    label_encode_dict[col] = dict(zip(dct.values(),dct.keys()))

label_encode_dict.keys()


In [ ]:
label_encode_dict['batting_team'] == label_encode_dict['bowling_team']

## 4.3: Save the encoded values to a json file

In [ ]:
with open(json_path, 'w') as f:
    json.dump(label_encode_dict, f)

## 4.4: Format the Dataset:
In this step I am trying to club the all rows with respect to matchID and Innings (as match ID is unique way to identify a particular match and Innings to identify who bat first).

Based on these two details, 
* grab all the batsmen and bowler details who batted and bowled in 1<sup>st</sup> 6-overs
* Calculate the total score (runs through bat + extra runs)
* How many players dismissed in 1st 6-overs


In [ ]:
print('### Shape of Dataframe before format_data : {}'.format(df_parsed.shape))
Runs_off_Bat_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['runs_off_bat'].sum()
Extras_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['extras'].sum()
TotalScore_6_overs = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['Total_score'].sum()

Total_WktsDown = df_parsed.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team'])['player_dismissed'].count()


In [ ]:
type(Runs_off_Bat_6_overs)

In [ ]:
bat_df = df_parsed.groupby(['match_id', 'venue', 'innings','batting_team', 'bowling_team'])['batsmen'].apply(list)
bow_df = df_parsed.groupby(['match_id', 'venue', 'innings','batting_team', 'bowling_team'])['bowlers'].apply(list)

now concat all these formated data into a single dataframe

In [ ]:
df_parsed.shape

In [ ]:
df_parsed = pd.DataFrame(pd.concat([bat_df, bow_df, Runs_off_Bat_6_overs, Extras_6_overs, TotalScore_6_overs, Total_WktsDown],axis=1)).reset_index()

if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path,'2_powerplay_summary.csv'),index=False)

## 4.5 : Align the batsmen and bowlers details in to a separate column

In above formated dataset, we got list of batsmen and bowlers details who batted and bowled in 6-overs.

Now we have to arrange these batsmen into a separate columns,
```
 * say bat1,bat2,bat3,bat4....bat10
 * say bow1,bow2,bow3.....bow6
```

Here I selected only 10-batsmen (as we have only 10-wickets), and 6-bowlers (can bowl in 6-overs) because in 6-overs this is only possible.

```
For proper prediction the order of batsmen and bowlers given the dataset matters. So we need to keep the order :
* batsmen : who batted 1st,2nd 3rd and 4th ... wicket same
* bowler  : who bowled in 1st,2nd,3rd,4th,5th and 6th overs same
```

## 4.6: Create a batsmen and bowlers dummy dataframe:
Inorder to keep track of the order same, so 1st I am going to create a dummy dataframe 
* with 10-batsmen with column names [bat1,bat2,.....bat9,bat10]
* with 6-bowlers with column names [bow1,bow2,bow3,bow4,bow5,bow6]

In [ ]:
df_parsed.shape

In [ ]:
bat  = pd.DataFrame(np.zeros((df_parsed.shape[0],10),dtype=float),columns=['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10'])
bowl = pd.DataFrame(np.zeros((df_parsed.shape[0],6),dtype=float),columns=['bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6'])

columns = ['bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10','bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']
df_bat_bow = pd.concat([bat,bowl],axis=1)
df_bat_bow



In [ ]:
df_parsed =  pd.concat([df_parsed, df_bat_bow], axis=1)
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path, '3_create_batsmen_bowler_df.csv'),index=False)

In [ ]:
df_parsed.head(3)

## 4.7: Update the batsmen list of elements into each column in the same order:

Means, data which is in list of elements in each matchID and innings into corresponding individual batsmen columns in the same order. 
```
Example, here in below list 1st batsmen is bat1 -> SC Ganguly, 2nd is bat2 -> BB McCullum, 3rd is bat3 -> RT Poting ...etc like 

"['SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'SC Ganguly', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'SC Ganguly', 'BB McCullum', 'SC Ganguly', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'RT Ponting', 'RT Ponting', 'RT Ponting', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'DJ Hussey', 'DJ Hussey', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'DJ Hussey', 'BB McCullum', 'Mohammad Hafeez', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'Mohammad Hafeez', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum', 'BB McCullum']"

```


In [ ]:
df_parsed['batsmen'] = df_parsed.batsmen.map(OrderedDict.fromkeys).apply(list)
df_parsed['bowlers'] = df_parsed.bowlers.map(OrderedDict.fromkeys).apply(list)
df_parsed.head()

In [ ]:
for row,val in enumerate(df_parsed.batsmen):
    for i in range(len(val)):
        df_parsed.loc[row,'bat%s'%(i+1)] = val[i]

for row,val in enumerate(df_parsed.bowlers):
    for i in range(len(val)):
        df_parsed.loc[row,'bow%s'%(i+1)] = val[i]

df_parsed.loc[:,['bat1','bat2','bat3','bat4','bat5','bat6','bat7','bat8','bat9','bat10','bow1','bow2','bow3','bow4','bow5','bow6']]

In [ ]:
if save_files:
    df_parsed.to_csv(Path.joinpath(dataset_path,'4_update_batsmen_bowler_column_names.csv'),index=False)
#df_parsed = pd.read_csv(Path.joinpath(dataset_path, '4_update_batsmen_bowler_column_names.csv'))
df_parsed.head()

In [ ]:
df_parsed.head(3)

So finally our dataframe is ready with batsmen and bowlers details.

So we can drop few columns which are not important.

In [ ]:
df_model = df_parsed[['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8','bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6', 'runs_off_bat', 'extras', 'Total_score', 'player_dismissed']]
df_model.columns

In [ ]:
def pandas_df_to_markdown_table(df):
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_csv(sep="|", index=False)))

pandas_df_to_markdown_table(df_model.head())

## 4.8: Encode the multiple Dtypes into single Dtype:

Now its time to use, the label encoded values (already done in previous steps) to encode the dataframe.

In [ ]:
json_path = Path.joinpath(dataset_path, 'label_encode.json')
with open(json_path) as f:
    data = json.load(f)

condition = False

for col in df_model.columns:
    if col in data.keys():
        condition = True
        col = col
    elif col in ['bat1', 'bat2','bat3', 'bat4', 'bat5', 'bat6', 'bat7', 'bat8', 'bat9', 'bat10']:
        condition = True
        col = 'Players' #'batsmen'
    elif col in ['bow1','bow2', 'bow3', 'bow4', 'bow5', 'bow6']:
        col = 'Players' #'bowlers'
        condition = True

    if condition:
        condition = False
        for key in data[col]:
            df_model = df_model.replace([key], data[col][key])

if save_files:
    df_model.to_csv(Path.joinpath(dataset_path, '5_model_df.csv'), index=False)

So finally dataset is ready. 
Next step is prepare model for training 

In [ ]:
pandas_df_to_markdown_table(df_model.head())

# **5. Normalization:**

convert entire dataframe into flot point numbers 

In [ ]:
df_model = df_model.applymap(np.float64)
df_model.info()

As you can see here initially our dataset is of size ~ 30MB, now its in 273 KB

In [ ]:
df_model.iloc[0:4,5:15]

In [ ]:
df_model.corr()

Here I am using **Min Max Scaling** to normalize the dataframe

In [ ]:
df_norm = (df_model - df_model.min())/(df_model.max() - df_model.min())


In [ ]:
df_norm.isna().sum()

In [ ]:
df_norm.fillna(0.0,inplace=True)

In [ ]:
df_norm[['bat9','bat10']]

In [ ]:
df_model.iloc[0:4,4:15]

# **6. Split Train & test data**

Split the dataset into 70% for train and 30% for test
![splitDatasetImg](https://miro.medium.com/max/1400/1*n7Ob33nRMq07BZPbNtfItw.png)

1. Identify the inputs vs targets from the pandas dataframe  and convert into Torch Tensor
2. Create the torch dataset
3. Now split the torch_ds into train_ds and test_ds datasets (70% vs 30%)
4. Create a dataloader for train_ds and test_ds

In [ ]:
inputs = torch.Tensor(df_norm.iloc[:,:-4].values.astype(np.float32))
targets = torch.Tensor(df_norm.loc[:,'Total_score'].values.reshape(-1,1).astype(np.float32))

In [ ]:
torch_ds = torch.utils.data.TensorDataset(inputs,targets)
len(torch_ds)

In [ ]:
train_ds_sz = int(round((len(torch_ds)*split_ratio[0])/100.0))
test_ds_sz  = len(torch_ds) - train_ds_sz

train_ds,test_ds = torch.utils.data.random_split(torch_ds,[train_ds_sz,test_ds_sz])
len(train_ds),len(test_ds)

In [ ]:
train_dl = torch.utils.data.DataLoader(dataset = train_ds,batch_size=train_bs, shuffle = True)
test_dl = torch.utils.data.DataLoader(dataset = test_ds,batch_size=2*train_bs)

# **7. MODEL Selection:**

Here I am using Linear model 

In [ ]:
model = torch.nn.Linear(inputs.shape[1],targets.shape[1])

model.weight.shape,model.bias.shape

## 7.1 Train & Test:

In [ ]:
from tqdm import tqdm

train_loss, test_loss = [],[]
def train(epoch,model,train_loader,optimizer,loss):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx,(data,target) in enumerate(pbar):
        optimizer.zero_grad()
        output = model(data)
        error = loss(output,target)
        error.backward()
        optimizer.step()
        #if epoch % 10 == 0:
        pbar.set_description(desc= f'Epoch : {epoch} : Train - loss={error.item():.4f} batch_id={batch_idx}')
    train_loss.append(error.item())


def test(epoch,model,test_loader,loss):
    model.eval()
    pbar = tqdm(test_loader)
    with torch.no_grad():
        for batch_idx,(data,target) in enumerate(pbar):
            output = model(data)
            error = loss(output,target)
            #if epoch %10 ==0:
            pbar.set_description(desc= f'Epoch : {epoch} : Test - loss={error.item():.4f} batch_id={batch_idx}')
        test_loss.append(error.item())


## 7.2 Loss & optimizer:

In [ ]:
loss = torch.nn.L1Loss()
opt  = torch.optim.SGD(model.parameters(),lr=lr) 

# **8. Train Model:**

In [ ]:
for epoch in range(epochs):
    train(epoch,model = model, train_loader = train_dl, optimizer = opt,loss = loss)
    test(epoch,model = model,  test_loader = test_dl, loss=loss)

## **8.1. Train vs Test Loss Graph:**

In [ ]:
plt.plot(train_loss)
plt.plot(test_loss);
plt.title('Train vs Test Loss');
plt.xlabel('epoch');
plt.ylabel('Loss');
plt.legend(['Train_Loss','Test_Loss']);
plt.savefig(pltName);

In [ ]:
'''
for i,j in test_dl:
    test_inp  = i[0]
    test_op   = j[0] 
    print(test_inp,test_op)
    break
'''

In [ ]:
test_inp = torch.Tensor([0.6562, 1.0000, 0.4286, 0.2857, 0.1336, 0.2767, 0.5742, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6360, 0.0511, 0.3145, 0.6621,
        0.0000, 0.0000])

test_op = torch.Tensor([0.4951])
actual    = (test_op*(df_model.Total_score.max() - df_model.Total_score.min() )) + df_model.Total_score.min()
model.eval()
pred     = (model(torch.Tensor(test_inp))*(df_model.Total_score.max() - df_model.Total_score.min() )) + df_model.Total_score.min()
print(f'Predicted Value : {pred.item()} vs Actual value : {actual.item()}')

# **9. Record metrics:**

In [ ]:
metrics = {
    'pltName'       : [pltName],
    'Model'         : ['nn.Linear(20,1)'],
    'Epochs'        : [epochs],
    'Loss Func'     : ['L1 Loss'],
    'Optimizer'     : ['SGD(model.parameters(),lr=lr)'],
    'LR'            : [lr],
    'Train vs Test BatchSz' : [(train_bs, 2*train_bs)],
    'Train Loss'    : [list(map(lambda x: float(f'{x:.4f}'), train_loss))],
    'Test Loss'     : [list(map(lambda x: float(f'{x:.4f}'), test_loss))],
    'Test Input'    : [test_inp],
    'Actual Score'  : [actual.item()],
    'Predicted'     : [pred.item()],
    'Notes'         : [changeDescription]

}


In [ ]:
if record:
    if Path.joinpath(Path.cwd(),recordedMetrics).exists():
        df = pd.read_csv(recordedMetrics)
        print('### Concatinating the Metrics to existing Dataframe')
        df_new = pd.concat([df,pd.DataFrame(metrics)],axis=0)
    else:
        print('### Created a new file')
        df_new = pd.DataFrame(metrics)

    df_new.to_csv(Path.joinpath(Path.cwd(),recordedMetrics),index=False)

In [ ]:
df = pd.read_csv(recordedMetrics,usecols=['Epochs','LR','Train Loss','Test Loss']) # 'Actual Score', 'Predicted'


In [ ]:
df.info()

In [ ]:
sns.regplot(x=df['Train Loss'],data = df)